In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Path to your text file
file_path = './harry_potter_dialogues_bilstm.txt'

with open(file_path, 'r') as file:
    lines = file.readlines()

# Buffer to hold processed lines
processed_lines = []

for line in lines:
    # Count the words in the current line
    word_count = len(line.split())

    # If the word count is 10 or less, keep the line
    if word_count <= 10:
        processed_lines.append(line)

# Join the processed lines back into a single string
result_text = ''.join(processed_lines)

print(result_text)

Harry:  Nearly, 
Harry:  Nothing, nothing . . . 

Vernon:  We could phone Marge, 
Petunia:  Don’t be silly, Vernon, she hates the boy. 
Vernon:  What about what’s-her-name, your friend — Yvonne? 
Petunia:  On vacation in Majorca, 
Harry:  You could just leave me here, 
Petunia:  And come back and find the house in ruins?  
Harry:  I won’t blow up the house, 
Petunia:  I suppose we could take him to the zoo, 

Vernon:  MOTORCYCLES DON’T FLY! 
Harry:  I know they don’t, It was only a dream. 

Harry:  What’s this? 
Petunia:  Your new school uniform, 

Vernon:  Get the mail, Dudley 
Dudley:  Make Harry get it. 
Harry:  Make Dudley get it. 
Vernon:  Poke him with your Smelting stick, 
Harry:  That’s mine! 
Vernon:  Who’d be writing to you? P-P-Petunia! 
Petunia:  Vernon! Oh my goodness — Vernon! 
Dudley:  I want to read that letter, 
Harry:  I want to read it, as it’s mine. 
Vernon:  Get out, both of you, 
Harry:  I WANT MY LETTER! 
Dudley:  Let me see it! 
Vernon:  Get out, 

Harry:  Why? 

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
# # Example dataset
# text = """
# Petunia: Up! Get up! Now! Up! Up! Are you up yet? Harry: Nearly, Petunia: Well, get a move on, I want you to look after the bacon. What did you say? Harry: Nothing, nothing . . .
# Petunia: Bad news, Vernon, Mrs. Figg’s broken her leg. She can’t take him. Now what? Vernon: We could phone Marge, Petunia: Don’t be silly, Vernon, she hates the boy. Vernon: What about what’s-her-name, your friend — Yvonne? Petunia: On vacation in Majorca, Harry: You could just leave me here, Petunia: And come back and find the house in ruins? Harry: I won’t blow up the house, Petunia: I suppose we could take him to the zoo, Vernon: That car’s new, he’s not sitting in it alone. . . . Petunia: Dinky Duddydums, don’t cry, Mummy won’t let him spoil your special day! Oh, good Lord, they’re here! Vernon: I’m warning you, I’m warning you now, boy — any funny business, anything at all — and you’ll be in that cupboard from now until Christmas.
# """

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([result_text])
total_words = len(tokenizer.word_index) + 1

# Convert text data into sequences of tokens
input_sequences = []
for line in result_text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
X, labels = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(f"Total Sequences: {len(input_sequences)}")
print(f"Max Sequence Length: {max_sequence_len}")
print(f"Total Words: {total_words}")


2024-04-18 07:58:04.599047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 07:58:13.384309: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/opt/conda/lib
2024-04-18 07:58:13.385106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl

Total Sequences: 34481
Max Sequence Length: 29
Total Words: 3694


In [5]:
# # Prepare the data for text file (similarly convert sequences back to text)
# sequences_text = []
# for sequence in input_sequences.tolist():
#     words = [tokenizer.index_word[i] for i in sequence if i != 0]  # Exclude padding (index 0)
#     sequences_text.append(' '.join(words))

# Save the sequences to a text file
with open('short_sequences.txt', 'w') as text_file:
        text_file.write(result_text)

# print("Saved shorter sequences to short_sequences.txt")


In [1]:
len(input_sequences[0])

NameError: name 'input_sequences' is not defined

In [6]:
max_sequence_len,total_words

(29, 3694)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    Bidirectional(LSTM(50, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(50)),
    Dropout(0.5),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


2024-04-18 07:58:20.101351: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-18 07:58:22.354403: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-18 07:58:22.358006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-18 07:58:22.367513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 100)           369400    
                                                                 
 bidirectional (Bidirectiona  (None, 28, 100)          60400     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 28, 100)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 100)              60400     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3694)              3

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)  # For example, 10% data as validation

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
checkpoint_path = r"./model_epoch_300.h5"
model = load_model(checkpoint_path)  # Adjust the epoch number as needed

1# Callbacks
checkpoint = ModelCheckpoint(
    'model_epoch_500_{epoch:02d}.h5',  # Saves each model with the epoch number
    save_best_only=False,  # Saves all models, not just the best one
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=500,
    batch_size=32,  # Smaller batch size for potentially better generalization
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],  # Only use the checkpoint callback
    verbose=1
)

Epoch 1/500
970/970 [==============================] - ETA: 0s - loss: 2.1981 - accuracy: 0.5318
Epoch 1: saving model to model_epoch_500_01.h5
970/970 [==============================] - 25s 21ms/step - loss: 2.1981 - accuracy: 0.5318 - val_loss: 12.9390 - val_accuracy: 0.1058
Epoch 2/500
967/970 [============================>.] - ETA: 0s - loss: 2.1887 - accuracy: 0.5319
Epoch 2: saving model to model_epoch_500_02.h5
970/970 [==============================] - 12s 13ms/step - loss: 2.1879 - accuracy: 0.5319 - val_loss: 13.0877 - val_accuracy: 0.1073
Epoch 3/500
967/970 [============================>.] - ETA: 0s - loss: 2.1965 - accuracy: 0.5283
Epoch 3: saving model to model_epoch_500_03.h5
970/970 [==============================] - 12s 12ms/step - loss: 2.1960 - accuracy: 0.5285 - val_loss: 13.1421 - val_accuracy: 0.1081
Epoch 4/500
967/970 [============================>.] - ETA: 0s - loss: 2.1985 - accuracy: 0.5318
Epoch 4: saving model to model_epoch_500_04.h5
970/970 [=============

In [ ]:

# Split the dataset into training and validation sets

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
# checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_small_new_new.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/model_epoch_{epoch:02d}.h5',  # Saves each model with the epoch number
    save_best_only=False,  # Saves all models, not just the best one
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=8,  # Smaller batch size for potentially better generalization
    validation_data=(X, y),
    callbacks=[checkpoint],  # Only use the checkpoint callback
    verbose=1
)
# # Train the model
# history = model.fit(X_train, y_train,
#                     epochs=300,
#                     batch_size=16,  # Smaller batch size for potentially better generalization
#                     validation_data=(X_val, y_val),
#                     callbacks=[early_stop, checkpoint],
#                     verbose=1)

Epoch 1/300
3879/3879 [==============================] - ETA: 0s - loss: 6.4321 - accuracy: 0.0401
Epoch 1: saving model to /content/drive/MyDrive/model_epoch_01.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3879/3879 [==============================] - 97s 23ms/step - loss: 6.4321 - accuracy: 0.0401 - val_loss: 6.1357 - val_accuracy: 0.0412
Epoch 2/300
3877/3879 [============================>.] - ETA: 0s - loss: 6.1394 - accuracy: 0.0477
Epoch 2: saving model to /content/drive/MyDrive/model_epoch_02.h5
3879/3879 [==============================] - 95s 25ms/step - loss: 6.1393 - accuracy: 0.0478 - val_loss: 5.9250 - val_accuracy: 0.0567
Epoch 3/300
3877/3879 [============================>.] - ETA: 0s - loss: 5.9906 - accuracy: 0.0564
Epoch 3: saving model to /content/drive/MyDrive/model_epoch_03.h5
3879/3879 [==============================] - 94s 24ms/step - loss: 5.9905 - accuracy: 0.0564 - val_loss: 5.7795 - val_accuracy: 0.0629
Epoch 4/300
3878/3879 [============================>.] - ETA: 0s - loss: 5.8641 - accuracy: 0.0625
Epoch 4: saving model to /content/drive/MyDrive/model_epoch_04.h5
3879/3879 [==============================] - 94s 24ms/step - loss: 5.8641 - accuracy: 0.0625 - val_l

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predictions, axis=-1)[0]  # Get the index with the highest probability

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Assuming 'tokenizer' and 'max_sequence_len' are already defined and initialized
# Example usage
print(generate_text("Petunia:  I suppose we could take him to the zoo,", 20, model, max_sequence_len, tokenizer))


Petunia:  I suppose we could take him to the zoo, zoo dementors anyway more before then beautiful beautiful won people for happen ” hagrid separate sum of away components than


In [13]:
import pickle

# Assuming 'tokenizer' is your tokenizer object
with open('./tokenizer_new.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
print("h")